In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from selenium.webdriver.common.by import By

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 1. Парсинг URL адресов с новостями (Selenium)

In [3]:
#Download driver using the link at the top of the notebook
chrome_driver_path = r"chromedriver_win32\chromedriver.exe"
links_filename = "links.txt"

In [38]:
categories = ["politics", "world", "economy", "society"]
categories_rus = ["Политика", 'В мире', "Общество", "Экономика"]
cat_translate = {"Политика":"politic", 
                 "Общество":"society", 
                 'В мире':"world", 
                 "Экономика":"economy"
                 }


In [5]:
# установка ublock чтобы это расширение было установлено по умолчанию

# options = Options()
# options.add_argument("user-data-dir=C:\\profile")
# driver = Chrome(chrome_options=options)
# driver.get("https://google.com")

In [8]:
# URL = r"https://lenta.ru/rubrics/world/"
URL= "https://ria.ru/"


def connect():
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=C:\\profile")
    chrome_options.add_argument('--disable-application-cache')
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
    driver.implicitly_wait(5)
    
    print(f"Current session is {driver.session_id}")
    # print("now time:", time.time()-start_time)
    # driver.get(url)
    # time.sleep(3)

    return driver


def get_links_from_one_page(driver, url):
    driver.get(url)
    time.sleep(5)
    
    recipe_links = []
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    button = driver.find_element(By.XPATH, "/html/body/div[10]/div[2]/div/div[4]/div/div[1]/div/div[3]")
    button.click()
    time.sleep(3)
                                    
    for i in range(1, 1300):
        
        if (i !=5):
            a = driver.find_element(By.XPATH, f'/html/body/div[10]/div[2]/div/div[4]/div/div[1]/div/div[2]/div[{i}]/div[1]/a[2]')
                                            # /html/body/div[10]/div[2]/div/div[4]/div/div[1]/div/div[2]/div[6]/div[1]/a[2]
                                            # /html/body/div[10]/div[2]/div/div[4]/div/div[1]/div/div[2]/div[21]/div[1]/a[2]
                                            # /html/body/div[10]/div[2]/div/div[4]/div/div[1]/div/div[2]/div[62]/div[1]/a[2]
        # print("now time:", time.time()-start_time)
        recipe_links.append(a.get_attribute("href"))
        if(i%10==0):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
    return recipe_links

# очистка файла с ссылками
with open(links_filename, "w") as file:
    pass

driver = connect()

# links = get_links_from_one_page(driver, URL)

for category in categories:
    # print("now time1:", time.time() - start_time)
    url = URL+category+"/"
    print(url)
    links = get_links_from_one_page(driver, url)
    # print("now time2:", time.time() - start_time)
    # driver.close()
    # print("now time3:", time.time() - start_time)
    # print(links)

    with open(links_filename, "a") as file:
        for link in links:
            file.write(link + "\n")

Current session is 1213761c19ecdc547e9487f1880ec9b3
https://ria.ru/politics/
https://ria.ru/world/
https://ria.ru/economy/
https://ria.ru/society/


## 2. Парсинг новостей (BeautifulSoup)

In [29]:
posts = []
with open(links_filename) as file:
    for line in file:
        posts.append(line[:-1])

In [33]:
posts[100]

'https://ria.ru/20230111/politubezhische-1844137535.html'

In [51]:
df = pd.DataFrame(columns=["article_id", "title", "category", "tags" , "text"])

In [52]:
index = 0 

for post in posts:
    # print(post)
    resp = requests.get(post)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    
    title = soup.title.text
    
    sep_position = title.find(" - ") 
    title = title[ : sep_position]
    # print(title)
    
    tags = soup.find_all("a", class_ = "article__tags-item")#.get_text()
    tags = [tag.get_text() for tag in tags]
    category = ''
    for cat in categories_rus:
        if cat in tags:
            category = cat_translate[cat]
            break
    if not category:    
        category = tags[0]
    tags = ",".join(tags)
    
    try:
        text = soup.find("div", class_ = "article__body js-mediator-article mia-analytics").get_text()
    except:
        continue
    
    df.loc[index] = [post, title, category,tags,  text] 
    index += 1
    if (index%500 == 0):
        print(f"{index}/5196")

500/5196
1000/5196
1500/5196
2000/5196
2500/5196


IndexError: list index out of range

In [53]:
# нет тегов поэтому я проигнорирую эту ссылку и продолжу с того места где остановился индекс
# спасибо хотя бы переменные не обнуляются при вознокновении ошибки
index

2802

In [57]:
index = 2803

for post in posts[index+1:]:
    # print(post)
    resp = requests.get(post)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    
    title = soup.title.text
    
    sep_position = title.find(" - ") 
    title = title[ : sep_position]
    # print(title)
    
    tags = soup.find_all("a", class_ = "article__tags-item")#.get_text()
    tags = [tag.get_text() for tag in tags]
    category = ''
    for cat in categories_rus:
        if cat in tags:
            category = cat_translate[cat]
            break
    if not category:    
        category = tags[0]
    tags = ",".join(tags)
    
    try:
        text = soup.find("div", class_ = "article__body js-mediator-article mia-analytics").get_text()
    except:
        continue
    
    df.loc[index] = [post, title, category,tags,  text] 
    index += 1
    if (index%500 == 0):
        print(f"{index}/5196")

3000/5196
3500/5196
4000/5196
4500/5196
5000/5196


In [58]:
df.drop(labels = [5200],axis = 0)

,article_id,title,category,tags,text
0,https://ria.ru/20230117/vsu-1845593558.html,Экс-советник Пентагона заявил о панике в ВСУ и...,politic,"Политика,Россия,Министерство обороны США,Воору...","МОСКВА, 17 янв — РИА Новости. ВСУ оказались в ..."
1,https://ria.ru/20230117/peregovory-1845587705....,В Киеве заявили об условиях для рассмотрения п...,politic,"Политика,Россия,Украина,Владимир Зеленский,Мих...","МОСКВА, 17 янв — РИА Новости. Украина не проти..."
2,https://ria.ru/20230117/pomosch-1845585234.html,В ГД внесут проект о совершенствовании правово...,politic,"Россия,Госдума РФ,Политика,Василий Пискарев,Ге...","МОСКВА, 17 янв – РИА Новости. В ближайшее врем..."
3,https://ria.ru/20230117/nato-1845581407.html,Express: Путин получил возможность потопить НА...,politic,"Политика,Владимир Путин,Россия,НАТО,Великобрит...","МОСКВА, 17 янв — РИА Новости. Россия может уст..."
4,https://ria.ru/20230117/nato-1845581407.html,Express: Путин получил возможность потопить НА...,politic,"Политика,Владимир Путин,Россия,НАТО,Великобрит...","МОСКВА, 17 янв — РИА Новости. Россия может уст..."
...,...,...,...,...,...
5590,https://ria.ru/20230111/elektromobil-184411236...,"""АвтоВАЗ"" может начать производство электромоб...",economy,"Экономика,Санкт-Петербург,АвтоВАЗ,Концерн ПВО ...","МОСКВА, 11 янв - РИА Новости. ""АвтоВАЗ"" готов ..."
5591,https://ria.ru/20230111/rubli-1844099434.html,Рубль усилил рост,economy,"Ситуация с курсами валют и ценами на нефть,Эко...","МОСКВА, 11 янв - РИА Новости. Рубль усилил рос..."
5592,https://ria.ru/20230111/neft-1844096324.html,Индии будет трудно отказаться от покупки нефти...,world,"Экономика,Индия,Россия,Urals,Brent,БКС Мир инв...","МОСКВА, 11 янв - РИА Новости. Индия особенно ч..."
5593,https://ria.ru/20230111/valyuta-1844093607.html,ЦБ возобновит операции на валютном рынке в рам...,economy,"Экономика,Россия,Центральный Банк РФ (ЦБ РФ),М...","МОСКВА, 11 янв — РИА Новости. Центробанк возоб..."


## 3. Обзор полученных данных

In [65]:
df.ilok[[5201]]

AttributeError: 'DataFrame' object has no attribute 'ilok'

In [43]:
df.describe()

,article_id,title,category,tags,text
count,24,24,24,24,24
unique,23,23,9,23,23
top,https://ria.ru/20230117/nato-1845581407.html,Express: Путин получил возможность потопить НА...,politic,"Политика,Владимир Путин,Россия,НАТО,Великобрит...","МОСКВА, 17 янв — РИА Новости. Россия может уст..."
freq,2,2,10,2,2


In [44]:
df.head()

,article_id,title,category,tags,text
0,https://ria.ru/20230117/vsu-1845593558.html,Экс-советник Пентагона заявил о панике в ВСУ и...,politic,"Политика,Россия,Министерство обороны США,Воору...","МОСКВА, 17 янв — РИА Новости. ВСУ оказались в ..."
1,https://ria.ru/20230117/peregovory-1845587705....,В Киеве заявили об условиях для рассмотрения п...,politic,"Политика,Россия,Украина,Владимир Зеленский,Мих...","МОСКВА, 17 янв — РИА Новости. Украина не проти..."
2,https://ria.ru/20230117/pomosch-1845585234.html,В ГД внесут проект о совершенствовании правово...,politic,"Россия,Госдума РФ,Политика,Василий Пискарев,Ге...","МОСКВА, 17 янв – РИА Новости. В ближайшее врем..."
3,https://ria.ru/20230117/nato-1845581407.html,Express: Путин получил возможность потопить НА...,politic,"Политика,Владимир Путин,Россия,НАТО,Великобрит...","МОСКВА, 17 янв — РИА Новости. Россия может уст..."
4,https://ria.ru/20230117/nato-1845581407.html,Express: Путин получил возможность потопить НА...,politic,"Политика,Владимир Путин,Россия,НАТО,Великобрит...","МОСКВА, 17 янв — РИА Новости. Россия может уст..."


## 4. Конвертирование в JSON формат

In [104]:
js = json.loads(df.to_json(force_ascii=False, orient='records'))

In [105]:
with open('news.json', 'w') as file:
    for i in js:
        file.write(json.dumps(i) + '\n')
